In [38]:
import pennylane as qml
from pennylane import numpy as np
import time as time
import torch
import torch.nn as nn


In [30]:
# We create our data and labels

x_train = np.array([0.2, 0.1, 0.2, 0.14, 0.11, 0.41, 0.55, 0.3, 0.31, 0.6])
y_train = np.array([1,1,1,1,1,-1,-1,-1,-1,-1])


# We define the circuit we are going to use

dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def simple_qubit_circuit(theta, inputs):
    qml.RX(inputs, wires=0)
    qml.RY(theta, wires=0)
    return qml.expval(qml.PauliZ(0))


In [7]:
err = 0
theta=0.2
t = time.time()
for x, y in zip(x_train, y_train):
    err += (simple_qubit_circuit(theta, x)-y)**2
print(err)
print(time.time()-t)

17.629756
0.01999950408935547


In [8]:
t = time.time()

print(sum((simple_qubit_circuit(theta,x_train)-y_train)**2))
print(time.time()-t)

17.732708
0.006994962692260742


In [84]:
import pennylane as qml
from pennylane import numpy as np
import time as time
import torch
import torch.nn as nn

dev = qml.device("default.qubit", wires=1)
@qml.qnode(dev, interface = 'torch')
def simple_qubit_circuit(inputs, theta):
    qml.RX(inputs, wires=0)
    qml.RY(theta, wires=0)
    return qml.expval(qml.PauliZ(0))
class QNet(nn.Module):
    def __init__(self):
        super().__init__()
        quantum_weights = np.random.normal(0, np.pi)
        self.quantum_weights = nn.parameter.Parameter(torch.tensor(quantum_weights,\
                                    dtype=torch.float32,requires_grad=True))
        shapes = {
            "theta": 1
        }
        self.q = qml.qnn.TorchLayer(simple_qubit_circuit, shapes)
        self.linear = nn.Linear(1,1)
    
    def forward(self, input_value):
        out = self.linear(input_value)
        print(out)
#         out = out.squeeze(1)
        return self.q(out)

In [85]:
# x_train = np.array([0.2, 0.1, 0.2, 0.14, 0.11, 0.41, 0.55, 0.3, 0.31, 0.6])
# x_train = torch.tensor(x_train)
x_train = torch.rand(1).to(torch.float64)
x_train = torch.atan(x_train)
model = QNet().double()
out = model(x_train)
out.mean().backward()

tensor([0.3765], dtype=torch.float64, grad_fn=<AddBackward0>)


RuntimeError: Function ExecuteTapesBackward returned an invalid gradient at index 0 - got [] but expected shape compatible with [1]

AttributeError: 'float' object has no attribute 'backward'

In [91]:
from qiskit import IBMQ
IBMQ.save_account('dbb6e66a619d886dd48e2e9a93c0d35197315b5a926524e6566ad0e6936274dce110f5fb433ad0d8200e54f0654cce0c7f1c20ba12fad19ac9f4acf1d0f6001d', overwrite=True)
IBMQ.load_account()


C:\Users\aksi01\AppData\Local\Temp\ipykernel_12504\2901611185.py:2: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  IBMQ.save_account('dbb6e66a619d886dd48e2e9a93c0d35197315b5a926524e6566ad0e6936274dce110f5fb433ad0d8200e54f0654cce0c7f1c20ba12fad19ac9f4acf1d0f6001d', overwrite=True)
C:\Users\aksi01\AppData\Local\Temp\ipykernel_12504\2901611185.py:2: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_p

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [23]:
dev = qml.device("qiskit.ibmq", wires=1, backend='ibmq_qasm_simulator', shots=1000)
@qml.qnode(dev, interface = 'torch')
def simple_qubit_circuit(inputs, theta):
    qml.RX(inputs, wires=0)
    qml.RY(theta, wires=0)
    return qml.expval(qml.PauliZ(0))
class QNet(nn.Module):
    def __init__(self):
        super().__init__()
        shapes = {
            "theta": 1
        }
        self.q = qml.qnn.TorchLayer(simple_qubit_circuit, shapes)
    
    def forward(self, input_value):
        return self.q(input_value)

In [25]:
x_train = np.array([0.2, 0.1, 0.2, 0.14, 0.11, 0.41, 0.55, 0.3, 0.31, 0.6])
x_train = torch.tensor(x_train)

model = QNet()
t1 = time.time()
out = model(x_train)
print("time taken for batch operations: ", time.time()-t1)
# out2 = []
# t2 = time.time()
# for x in x_train:
#     out2.append(model(x).item())
# print("time taken for sequential operations: ", time.time()-t2)

print(out)
# print(out2)

time taken for batch operations:  9.492548942565918
tensor([0.9400, 0.9640, 0.9400, 0.9640, 0.9800, 0.9040, 0.8120, 0.9360, 0.9160,
        0.8080], dtype=torch.float64, grad_fn=<SqueezeBackward0>)


In [53]:
import math
import pennylane as qml
import torch
import torch.nn as nn
from torch.distributions import Categorical
import torch.nn.functional as F
from torch import optim

def encode(n_qubits, inputs):
    for wire in range(n_qubits):
        qml.RX(inputs[:,3 * wire + 0], wires=wire)
        qml.RY(inputs[:,3 * wire + 1], wires=wire)
        qml.RZ(inputs[:,3 * wire + 2], wires=wire)

# At the moment contains only two rotation gates Ry and Rz
def ansatz_1(n_qubits, y_weight, z_weight):
    for wire, y_weight in enumerate(y_weight):
        qml.RY(y_weight, wires=wire)
    for wire, z_weight in enumerate(z_weight):
        qml.RZ(z_weight, wires=wire)
    for wire in range(n_qubits):
        qml.CZ(wires=[wire, (wire + 1) % n_qubits])


def get_model(n_qubits=2, n_layers=1, num_sub_layer, ansatz=1):
    dev = qml.device("default.qubit", wires=n_qubits)
    data_per_qubit = 3
    shapes = {
        "y_weights": (n_layers, num_sub_layer, n_qubits),
        "z_weights": (n_layers, num_sub_layer, n_qubits),
    }

    @qml.qnode(dev, interface='torch')
    def circuit(inputs, y_weights, z_weights):

        def sublayer(inputs, sub_layer_idx, layer_idx, y_weights, z_weights):
            encode(n_qubits, inputs[:, sub_layer_idx *data_per_qubit*n_qubits:(sub_layer_idx+1) *data_per_qubit*n_qubits])
            ansatz_1(n_qubits, y_weights[layer_idx][sub_layer_idx], z_weights[layer_idx][sub_layer_idx])
        for layer_idx in range(n_layers):
            for sub_layer_idx in range(num_sub_layer):
                sublayer(inputs, sub_layer_idx, layer_idx, y_weights, z_weights)

        # return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]
        # TODO: Fix me
        return qml.expval(qml.PauliZ(wires=0)), qml.expval(qml.PauliZ(wires=1))
    model = qml.qnn.TorchLayer(circuit, shapes)
    return model, dev


class QuantumNet(nn.Module):
    def __init__(self, n_layers, ansatz, num_sub_layer, n_qubits):
        super(QuantumNet, self).__init__()
        self.n_qubits = n_qubits
        self.n_actions = 3

        self.q_layers, self.dev = get_model(n_qubits=self.n_qubits,
                                  n_layers=n_layers, num_sub_layer=num_sub_layer, ansatz=ansatz)
        self.out_layer = nn.Linear(self.n_qubits, 1)
        self.tanh = nn.Tanh()

    def forward(self, inputs):
        inputs = torch.atan(inputs).to(torch.float64) #(batch_size x 36)
        outputs = self.q_layers(inputs).to(torch.float32)
        outputs = self.out_layer(outputs)
        return outputs


<Quantum Torch Layer: func=circuit>


In [58]:
x_train = torch.rand(10, 12).to(torch.float64)
y_train = torch.rand(10, 1).to(torch.float64)
out = model(x_train)
loss = (y_train-out).mean()
optimiser = optim.AdamW(model.parameters(), lr=0.0001)
optimiser.zero_grad()
loss.backward()
optimiser.step()




In [67]:
class SharedNetwork(nn.Module):
    def __init__(self, hidden_dim=256):
        super(SharedNetwork, self).__init__()

        # input_shape = [None, 400, 400, 3]
        # self.inp_norm = nn.LayerNorm([400, 400])
        self.inp_norm = nn.InstanceNorm2d(3)

        self.conv1 = nn.Conv2d(3, 32, kernel_size=(8, 8), stride=(4, 4))
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
        self.layer_norm2 = nn.LayerNorm([64, 48, 48])
        self.conv3 = nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
        self.conv4 = nn.Conv2d(64, 128, kernel_size=(9, 9), stride=(3, 3))
        self.layer_norm4 = nn.LayerNorm([128, 13, 13])
        self.conv5 = nn.Conv2d(128, 128, kernel_size=(9, 9), stride=(1, 1))
        self.conv6 = nn.Conv2d(128, hidden_dim, kernel_size=(5, 5), stride=(1, 1))
        nn.init.orthogonal_(self.conv1.weight, np.sqrt(2))
        nn.init.orthogonal_(self.conv2.weight, np.sqrt(2))
        nn.init.orthogonal_(self.conv3.weight, np.sqrt(2))
        nn.init.orthogonal_(self.conv4.weight, np.sqrt(2))
        nn.init.orthogonal_(self.conv5.weight, np.sqrt(2))
        nn.init.orthogonal_(self.conv6.weight, np.sqrt(2))

        self.fc1 = nn.Linear(hidden_dim, hidden_dim, bias=True)
        self.fc_norm1 = nn.LayerNorm([hidden_dim])
        nn.init.orthogonal_(self.fc1.weight, np.sqrt(2))
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.recurrent_layer = nn.LSTM(hidden_dim+4, hidden_dim, batch_first=True)
        for name, param in self.recurrent_layer.named_parameters():
            if "bias" in name:
                nn.init.constant_(param, 0)
            elif "weight" in name:
                nn.init.orthogonal_(param, np.sqrt(2))
    def forward(self, obs:torch.tensor, recurrent_cell:torch.tensor, cat_tensor: torch.tensor=None, sequence_length:int=1):
        batch_size = obs.size()[0]
        obs = self.inp_norm(obs)
        x = self.relu(self.conv1(obs))
        x = self.relu(self.layer_norm2(self.conv2(x)))
        x = self.relu(self.conv3(x))
        x = self.relu(self.layer_norm4(self.conv4(x)))
        x = self.relu(self.conv5(x))
        # print(x.size())
        x = self.relu(self.conv6(x))
        # x = self.pool(x)
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = x.reshape((batch_size, -1))
        x = self.relu(self.fc_norm1(self.fc1(x)))
        x = torch.cat((x, cat_tensor), dim=1)
        if sequence_length == 1:
            # Case: sampling training data or model optimization using sequence length == 1
            x, recurrent_cell = self.recurrent_layer(x.unsqueeze(1), recurrent_cell)
            x = x.squeeze(1)  # Remove sequence length dimension
        else:
            # Case: Model optimization given a sequence length > 1
            # Reshape the to be fed data to batch_size, sequence_length, data
            x_shape = tuple(x.size())
            x = x.reshape((x_shape[0] // sequence_length), sequence_length, x_shape[1])

            # Forward recurrent layer
            x, recurrent_cell = self.recurrent_layer(x, recurrent_cell)

            # Reshape to the original tensor size
            x_shape = tuple(x.size())
            x = x.reshape(x_shape[0] * x_shape[1], x_shape[2])

        return x, recurrent_cell


In [68]:
class Q_PPO(nn.Module):
    def __init__(self, n_layers=1, ansatz=1, n_qubits=2, hidden_dim=12):
        super().__init__()
        self.shared_network = SharedNetwork(hidden_dim=hidden_dim)
        self.hidden_dim = hidden_dim
        self.bw_per_sublayer = n_qubits*3
        self.num_sub_layer = math.ceil(hidden_dim/self.bw_per_sublayer)
        self.value_network = QuantumNet(n_layers, ansatz, num_sub_layer=self.num_sub_layer,n_qubits=n_qubits)


    def forward(self, obs, recurrent_cell, cat_tensor=None, sequence_length:int=1):
        obs =obs.permute(0, 3, 1, 2) # 1x3x400x400
        value = None
        features, (hx, cx) = self.shared_network(obs=obs, recurrent_cell=recurrent_cell, sequence_length=sequence_length, cat_tensor=cat_tensor) # features = 1x32
        # Pads the hidden_dim vector
        if self.bw_per_sublayer * self.num_sub_layer != self.hidden_dim:
            #     Need to do input padding for value_network
            padding_len = self.bw_per_sublayer * self.num_sub_layer - self.hidden_dim
            features_value = torch.cat((features, torch.zeros((features.shape[0], padding_len)).to(features.device)), dim=1)
            value = self.value_network(features_value)
        else:
            value = self.value_network(features)
        return value, (hx, cx)


In [73]:
PPO_model = Q_PPO()
x_train = torch.rand(10, 400, 400, 3)
y_train = torch.rand(10, 1)
recurrent_cell = (torch.rand(1,2,12), torch.rand(1,2,12))
cat_tensor = torch.rand(10, 4)
y_out, _ = PPO_model(x_train, recurrent_cell=recurrent_cell, cat_tensor=cat_tensor, sequence_length=5)
loss = (y_out - y_train).mean()
# loss.backward()

<Quantum Torch Layer: func=circuit>


In [65]:
import math
import pennylane as qml
import torch
import torch.nn as nn
import numpy as np
dev = qml.device("default.qubit", wires=1)
@qml.qnode(dev, interface = 'torch')
def simple_qubit_circuit(inputs, theta):
    qml.RX(inputs, wires=0)
    qml.RY(theta, wires=0)
    return qml.expval(qml.PauliZ(0))

class QNet(nn.Module):
    def __init__(self):
        super().__init__()
        quantum_weights = np.random.normal(0, np.pi)
        shapes = {
            "theta": 1
        }
        self.q = qml.qnn.TorchLayer(simple_qubit_circuit, shapes)
    
    def forward(self, input_value):
        return self.q(input_value)

class Q_PPO1(nn.Module):
    def __init__(self):
        super().__init__()
        self.shared_network = nn.Linear(5, 1)
        self.value_network = QNet()

    def forward(self, obs):
        features = self.shared_network(obs)   
#         features = features.to(torch.float64)
        value = self.value_network(features)
        return value


In [57]:
PPO_model = Q_PPO1().double()
x_train = torch.rand(10, 5).to(torch.float64)
y_train = torch.rand(1)
y_out = PPO_model(x_train)
loss = (y_out - y_train).mean()
loss.backward()

RuntimeError: Function ExecuteTapesBackward returned an invalid gradient at index 0 - got [] but expected shape compatible with [10]

In [69]:
PPO_model = Q_PPO1().double()
x_train = torch.tensor([0.1, 0.2, 0.45, 0.123, 0.76]).to(torch.float64)
y_train = torch.rand(1)
y_out = PPO_model(x_train)
loss = (y_out - y_train).mean()
loss.backward()

RuntimeError: Function ExecuteTapesBackward returned an invalid gradient at index 0 - got [] but expected shape compatible with [1]